In [ ]:
import math
import pdfkit
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
browser=webdriver.Chrome("chromedriver.exe")
browser.get("https://www.fcps.edu/")
browser.maximize_window()

In [ ]:
#browser.find_element_by_xpath('//a[contains(@href,"/schools-centers")]').click()
link=browser.find_element(By.XPATH  , '//*[@id="mm-0"]/div/div[2]/header/div/ul/li[4]/a').get_attribute("href")
browser.get(link)

In [ ]:
browser.find_element_by_id('edit-items-per-page').click()
browser.find_element_by_xpath('//*[@id="edit-items-per-page"]/option[3]').click()

In [ ]:
browser.find_element_by_id('block-school-center-school-type').click()

In [ ]:
#browser.find_element_by_id('school-center-school-type-74').click()
browser.find_element_by_id('school-center-school-type-77').click()
#browser.find_element_by_id('school-center-school-type-75').click()

In [ ]:
records=browser.find_element_by_xpath('//*[@id="block-school-center-school-type"]/div/div/ul/li[1]/label/span[2]').get_attribute('innerText')

In [ ]:
browser.find_element_by_xpath('//*[@id="main"]/div/div[2]/div/div/div[2]/div[2]/div[2]/div/a').click()

In [ ]:
def scroll_down(elem, num,driver):
    for _ in range(num):
        time.sleep(0.5)
        try:
            driver.find_element_by_class_name('pager')
            return
        except:
            elem.send_keys(Keys.PAGE_DOWN)

In [ ]:
def scroll_again(driver):
    SCROLL_PAUSE_TIME = 0.1
    elem = driver.find_element_by_tag_name("body")
    prev_height = elem.get_attribute("scrollHeight")


    for i in range(0, 100):

        scroll_down(elem,50,driver)
        print("waiting")

        time.sleep(SCROLL_PAUSE_TIME)

        
        if elem.get_attribute("scrollHeight") == prev_height and i > 0:
            break
        prev_height = elem.get_attribute("scrollHeight")

In [ ]:
pages_total=math.ceil(int(records[1:len(records)-1])/50)
pages_total

In [ ]:
p=0
l=[]
addr=[]
email=[]
pname=[]
while p<pages_total:
    scroll_again(browser)
    schools = browser.find_elements_by_class_name("view__row")
    for i in schools:
    #l.append(a.text.split('\n')[0])
        blocks=i.find_elements_by_class_name("node__section")
        c=0
        for q in blocks:
            if(c<3):
                if(c==0):
                    l.append(q.find_element_by_tag_name('a').get_attribute('href'))
                elif(c==1):
                    addr.append(q.find_element_by_tag_name('a').get_attribute('href'))
                else:
                    email.append(q.find_element_by_tag_name('a').get_attribute('href').split(":")[1])
                    pname.append(q.find_element_by_tag_name('a').get_attribute('innerText').split(",")[0])
                    #print(q.find_element_by_tag_name('a').get_attribute('innerText'))
                c=c+1
            else:
                break
        
    p=p+1
    try:
        browser.find_element_by_class_name("pager__item--next").click()
    except:
        print('No')
    

In [ ]:
pname

In [ ]:
l[0].split("/").pop(-1)

In [ ]:
browser.close()

In [ ]:
school_name=[]
staff_links=[]

In [ ]:
for i in l:
    browser=webdriver.Chrome("chromedriver.exe")
    browser.get(i)
    school_name.append(i.split("/").pop(-1))
    #staff_links.append(browser.find_element_by_xpath('//*[@id="main"]/div/article/div/div[2]/div/div[1]/a[4]').get_attribute('href'))
    staff_links.append(browser.find_element_by_xpath ('//a[contains( text( ), "Staff Directory")]').get_attribute('href'))
    #print(staff_links[0])
    browser.close()

In [ ]:
list_of_schools= pd.DataFrame(columns =['Name' , 'School Link', 'Staff List Link','Principal Name','Principal email','Address'])
list_of_schools['Name']=school_name
list_of_schools['School Link']=l
list_of_schools['Staff List Link']=staff_links
list_of_schools['Principal Name']=pname
list_of_schools['Principal email']=email
list_of_schools['Address']=addr

In [ ]:
for j in range(0,len(l)):
    #time.sleep(2)
    job=webdriver.Chrome("chromedriver.exe")
    job.get(list_of_schools['Staff List Link'][j])
    job.maximize_window()
    list_names=int(job.find_element_by_xpath('//h2[contains( text( ), "Results")]').text.split().pop(-1))
    staff_pages_total=math.ceil(list_names/50)
    #staff_pages_total
    job.find_element_by_id('edit-items-per-page').click()
    job.find_element_by_xpath('//*[@id="edit-items-per-page"]/option[3]').click()
    p=0
    db= pd.DataFrame(columns =['Name' , 'Position'])
    N=[]
    P=[]
    while p<staff_pages_total:
        scroll_again(job)
        schools = job.find_elements_by_tag_name("tr")
        trt=0
        for i in schools:
        #l.append(a.text.split('\n')[0])
            if(trt==0):
                trt=trt+1
                continue
            nama=i.find_element_by_class_name("views-field-field-last-name").get_attribute("innerText")
            print(nama)
            N.append(nama.split(',')[1]+' '+nama.split(',')[0])
            P.append(i.find_element_by_class_name("views-field-field-staff-title").get_attribute("innerText"))

        p=p+1
        try:
            job.find_element_by_class_name("pager__item--next").click()
        except:
            print('No')
    db['Name']=N
    db['Position']=P
    namei=list_of_schools['Name'][j]+'.csv'
    #print(namei)
    db.to_csv(namei)
    f = open(list_of_schools['Name'][j]+'.html','w')
    a = db.to_html()
    f.write(a)
    f.close()
    #time.sleep(1)
    job.close()

In [ ]:
len(P)

In [ ]:
list_of_schools.to_csv('Fairfax_County_Public_Schools.csv')
f = open('Fairfax_County_Public_Schools.html','w')
a = list_of_schools.to_html()
f.write(a)
f.close()

In [ ]:
list_of_schools